# ***0. Import libraries***

In [1]:
import re
import gensim
from gensim.utils import simple_preprocess
import jieba
import pandas as pd

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## ***1. Remove spaces between text***

In [2]:
df = pd.read_csv('data/leavehomesafe.csv', encoding="utf-8-sig")
df['content'] = df['thread_title'] + df['msg']
df

,ID,thread_id,thread_title,category,forum,stance,msg,reply_time,reply_date,day,content
0,HD_29534436,29534436,安心出行app下周一出爐 毋須記名及不使用gps記錄,時事新聞,hkdiscuss,blue,創新及科技局局長薛永恒宣布，「安心出行」應用程式下周一供市民下載，自行紀錄用家出行習慣，方便...,2020-11-11 11:56,2020-11-11,1,安心出行app下周一出爐 毋須記名及不使用gps記錄創新及科技局局長薛永恒宣布，「安心出行」...
1,HD_29534444,29534444,安心出行下周啟用助市民記錄行程場所爆疫會有通知,時事新聞,hkdiscuss,blue,"用唔用都可以照出入場所及乘坐交通工具，有咩人會用呀？,睇吓你去過邊度飲酒，邊度揼骨？,政府出...",2020-11-11 12:07,2020-11-11,1,安心出行下周啟用助市民記錄行程場所爆疫會有通知用唔用都可以照出入場所及乘坐交通工具，有咩人會...
2,HG_7327032,7327032,安心出行流動應用程式下周一起下載六千場地可使用,CA,hkgolden,yellow,"齊揮手.jpg ,十年前，唔會講｢出行｣又係支共文化入侵,想知個apps有乜用雖然都唔會do...",2020-11-11 12:29:14,2020-11-11,1,"安心出行流動應用程式下周一起下載六千場地可使用齊揮手.jpg ,十年前，唔會講｢出行｣又係支..."
3,HG_7327159,7327159,港府下週一推安心出行app 稱屬自願參與用戶無須登記姓名,CA,hkgolden,yellow,"代言人,尖尖尖尖尖～～～～～尖尖尖～～～尖～～～,安心上路,如果doo一次有錢收一定大把人doo",2020-11-11 14:38:24,2020-11-11,1,"港府下週一推安心出行app 稱屬自願參與用戶無須登記姓名代言人,尖尖尖尖尖～～～～～尖尖尖～..."
4,HD_29536981,29536981,政府推出安心出行流動應用程式,消閒娛樂,hkdiscuss,blue,政府今日（十一月十一日）公布推出感染風險通知「安心出行」流動應用程式（leavehomesa...,2020-11-12 02:13,2020-11-12,2,政府推出安心出行流動應用程式政府今日（十一月十一日）公布推出感染風險通知「安心出行」流動應用...
...,...,...,...,...,...,...,...,...,...,...,...
3689,BB_23578280,23578280,老人家安心出行手機,自由講場,babykingdom,blue,"老人家用唔慣智能電話， 好多時要撳好多次先撳到number, 撳好耐先打到電話用到發忟。 想...",2022-05-09 21:45:00,2022-05-09,545,"老人家安心出行手機老人家用唔慣智能電話， 好多時要撳好多次先撳到number, 撳好耐先打到..."
3690,HG_7560717,7560717,而家係咪去酒店賓館爆房都要嘟安心打兩針,CA,hkgolden,yellow,"首先你要有仔, ,高登仔唔去爆房",2022-05-09 12:26:34,2022-05-09,545,"而家係咪去酒店賓館爆房都要嘟安心打兩針首先你要有仔, ,高登仔唔去爆房"
3691,BB_23578547,23578547,如果今日有更新個安心出行,自由講場,babykingdom,blue,"你哋盡快check下你哋個針卡仲喺唔喺度 ， 因為今日好多人都冇咗個針卡要重新入過。,還在無...",2022-05-10 21:16:00,2022-05-10,546,如果今日有更新個安心出行你哋盡快check下你哋個針卡仲喺唔喺度 ， 因為今日好多人都冇咗個...
3692,BB_23578570,23578570,緊急 安心出行更新,自由講場,babykingdom,blue,andro機，3.2.5更新版本 會無咗針咭紀錄 裝咗醫健通嘅可以滙入 無裝嘅最好影低針紙個...,2022-05-10 23:04:00,2022-05-10,546,緊急 安心出行更新 andro機，3.2.5更新版本 會無咗針咭紀錄 裝咗醫健通嘅可以滙入 ...


In [3]:
def sent_to_words_space(sentences):
    for sent in sentences:
        sent = sent.lower()
        sent = re.sub("[~^0-9]", "", sent)
        sent = re.sub(r'http\S+', '', sent)
        sent = re.sub(r'https\S+', '', sent)
        sent = re.sub('<[^>]+>', '', sent)
        sent = re.sub('<[^<]+?>', '', sent)
        sent = re.sub('\S*@\S*\s?', '', sent)
        sent = re.sub('\s+', ' ', sent)
        sent = re.sub("\'", "", sent)
        sent = re.sub(r'[^\w\s]', ' ', sent)
        sent = sent.replace('/>', '')
        sent = sent.replace('分享自lihkg討論區', '')
        sent = sent.replace('向左走', '')
        sent = sent.replace('向右走', '')
        sent = sent.replace('mdash', '')
        sent = sent.replace('&hellip', '')
        sent = sent.replace('&quot', '')
        sent = sent.replace('&quot;', '')
        sent = sent.replace('quot', '')
        sent = sent.replace('&ldquo', '')
        sent = sent.replace('&lsquo', '')
        sent = sent.replace('&rdquo', '')
        sent = sent.replace('&rsquo', '')
        sent = sent.replace('&gt', '')
        sent = sent.replace('id', '')
        sent = sent.replace('src', '')
        sent = sent.replace('assets', '')
        sent = sent.replace('faces', '')
        sent = sent.replace('lm2', '')
        sent = sent.replace('gif', '')
        sent = sent.replace('class', '')
        sent = sent.replace('id', '')
        sent = sent.replace('hkgmoji', '')
        sent = sent.replace('\n', ' ')
        sent = sent.replace(' |', '. ')
        sent = sent.replace("?amp;", "&")

        yield sent

In [4]:
df['content'] = df['content'].astype(str)

msg_space = df.content.values.tolist()

msg_space_words = list(sent_to_words_space(msg_space))

print(msg_space_words[:1])

['安心出行app下周一出爐 毋須記名及不使用gps記錄創新及科技局局長薛永恒宣布  安心出行 應用程式下周一供市民下載 自行紀錄用家出行習慣 方便日後追蹤 減低社區傳播風險 用家進入參與計劃的場  安心出行 app下周一出爐 毋須記名及不使用gps記錄   希望市民齊心支持 安心出行 app 市民萬一出事也可讓政府部門追查源頭及相關地點 這做法跟內地也相同都是有效追查病者行蹤嘅方法 新聞來源連結  有撚用咩  港台事件曱甴話公眾利益凌駕私隱架咩  快d用健康碼才是 根本唔使理            照裝gps 講明有gps      唔想裝既咪能裝lor 你個死佬  尋日又去搵個隻狐狸精   薛永恆食得屎 有gps唔用而係每行一步掃描二維碼一次 掃描之後咪一樣有記錄 我就唔會用  呢d中央共狗野 唔會裝 手動掃描     唔記名無記錄    有鴨同冇都無分別 吊  晒錢晒時間 自欺欺人 google已經記錄晒我既行蹤 一開google map已經睇晒 我唔覺得好多人會用囉    呢回鬼門關大開 你唔好將呢d垃圾香港apps同中共扯上關係  影衰中共 中共既防疫追蹤系統好過香港千倍 唔駛審 又會話d資料send大陸了 香港疫情冇得斷尾 好大原因是 黃暴政治 乃共好丫  起碼唔使中病毒 乃西就慘啦  弗系抗疫  仲要自帶狗糧  用完即棄 冇得斷因為武漢放毒 你明唔明 健康碼要加強效用就要加入 安心出行 呢種功能  武漢放毒肺炎正仆街 你講乜都唔會改變中國抗疫成功  老美日日創新高呢個事實 如果閣下只有呢d技倆  就恕我失陪 我對化膿功  膠登腦殘果類廢up  已免疫 武漢肺炎 全世界人都討厭中國 全世界都可以唔like中國   唯獨只有港澳區嘅市民永遠不行 水 糧 電 日用品都要靠內地同胞生產送來 唔好再講要比錢去買  我們香港人有什麼資格偏低或討厭國家  自知量力 唔好再裝大香港 人講你又講咁冇腦 嘥鳩氣  自願性 手動掃描  當事人選擇性掃的行蹤記錄和有心隱瞞的口述有什麼分別   政府高官始終改唔到除褲放屁的思維 唔知有冇搵許志安同黃心穎做宣傳  安心出行 相信唔包曱甴   最仆街都係美國肺炎害全球 大陸bb飲紐西蘭荷蘭奶粉都係外國送來讓大 唔好再講要比錢去買  中國人有咩資格唔飲奶思源睇下邊個啲奶養大你  大陸人用嘅電腦 ipad軟件都係美國人研發送來 唔好再講

In [5]:
df['content'] = msg_space_words
df.to_csv("results/1_PP.csv", index=False, encoding="utf-8")
df.head(5)

,ID,thread_id,thread_title,category,forum,stance,msg,reply_time,reply_date,day,content
0,HD_29534436,29534436,安心出行app下周一出爐 毋須記名及不使用gps記錄,時事新聞,hkdiscuss,blue,創新及科技局局長薛永恒宣布，「安心出行」應用程式下周一供市民下載，自行紀錄用家出行習慣，方便...,2020-11-11 11:56,2020-11-11,1,安心出行app下周一出爐 毋須記名及不使用gps記錄創新及科技局局長薛永恒宣布 安心出行 ...
1,HD_29534444,29534444,安心出行下周啟用助市民記錄行程場所爆疫會有通知,時事新聞,hkdiscuss,blue,"用唔用都可以照出入場所及乘坐交通工具，有咩人會用呀？,睇吓你去過邊度飲酒，邊度揼骨？,政府出...",2020-11-11 12:07,2020-11-11,1,安心出行下周啟用助市民記錄行程場所爆疫會有通知用唔用都可以照出入場所及乘坐交通工具 有咩人會...
2,HG_7327032,7327032,安心出行流動應用程式下周一起下載六千場地可使用,CA,hkgolden,yellow,"齊揮手.jpg ,十年前，唔會講｢出行｣又係支共文化入侵,想知個apps有乜用雖然都唔會do...",2020-11-11 12:29:14,2020-11-11,1,安心出行流動應用程式下周一起下載六千場地可使用齊揮手 jpg 十年前 唔會講 出行 又係支...
3,HG_7327159,7327159,港府下週一推安心出行app 稱屬自願參與用戶無須登記姓名,CA,hkgolden,yellow,"代言人,尖尖尖尖尖～～～～～尖尖尖～～～尖～～～,安心上路,如果doo一次有錢收一定大把人doo",2020-11-11 14:38:24,2020-11-11,1,港府下週一推安心出行app 稱屬自願參與用戶無須登記姓名代言人 尖尖尖尖尖 尖尖尖 ...
4,HD_29536981,29536981,政府推出安心出行流動應用程式,消閒娛樂,hkdiscuss,blue,政府今日（十一月十一日）公布推出感染風險通知「安心出行」流動應用程式（leavehomesa...,2020-11-12 02:13,2020-11-12,2,政府推出安心出行流動應用程式政府今日 十一月十一日 公布推出感染風險通知 安心出行 流動應用...


## ***2. Preliminary dictionary-based tokenization***

In [6]:
df = pd.read_csv("results/1_PP.csv", encoding='utf-8-sig')
df.head(1)

,ID,thread_id,thread_title,category,forum,stance,msg,reply_time,reply_date,day,content
0,HD_29534436,29534436,安心出行app下周一出爐 毋須記名及不使用gps記錄,時事新聞,hkdiscuss,blue,創新及科技局局長薛永恒宣布，「安心出行」應用程式下周一供市民下載，自行紀錄用家出行習慣，方便...,2020-11-11 11:56,2020-11-11,1,安心出行app下周一出爐 毋須記名及不使用gps記錄創新及科技局局長薛永恒宣布 安心出行 ...


In [7]:
df['content'] = df['content'].astype(str)
data = df['content'].values.tolist()
print("Total Number of Documents:", len(data))

Total Number of Documents: 3694


In [8]:
jieba.load_userdict("dict/dict.txt")


def stopwordslist():
    stopwords = [line.strip() for line in open('dict/canton.txt', 'r', encoding='utf-8').readlines()]
    return stopwords


# (Optional) 只要中文
def is_chinese(uchar):
    if uchar >= '\u4e00' and uchar <= '\u9fa5':
        return True
    else:
        return False


# 定義停詞函數 對句子進行中文分詞
def seg_depart(sentence):
    # 對文檔中的每一行進行中文分詞
    sentence_depart = jieba.cut(sentence.strip())
    # 輸出結果為outstr
    stopwords = stopwordslist()
    outstr = ''
    # 去停用詞
    for word in sentence_depart:
        if word not in stopwords and (len(word.strip()) > 0) and is_chinese(word):
            outstr += word
            outstr += " "
    return outstr


# 分詞後的結果
result_fenci = []
for i in data:
    result_fenci.append([i, seg_depart(i)])

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/6k/lvj277mx6834z6rvr_nxth_00000gn/T/jieba.cache
Loading model cost 0.464 seconds.
Prefix dict has been built successfully.


In [9]:
result_fenci = [i[1].split(' ')[:-1] for i in result_fenci]
print(result_fenci[:5])

[['安心出行', '下周一', '出爐', '毋須', '記名', '及', '不', '使用', '記錄', '創新', '及', '科技', '局局長', '薛', '永恒', '宣布', '安心出行', '應用程式', '下周一', '供', '市民', '下載', '自行', '紀錄', '用家', '出行', '習慣', '方便', '日後', '追蹤', '減低', '社區', '傳播', '風險', '用家', '進入', '參與', '計劃', '的場', '安心出行', '下周一', '出爐', '毋須', '記名', '及', '不', '使用', '記錄', '希望', '市民', '齊心', '支持', '安心出行', '市民', '萬一出', '事', '也', '可讓', '政府部門', '追查', '源頭', '及', '相關', '地點', '這', '做法', '跟', '內地', '也', '相同', '都是', '有效', '追查', '病者', '行蹤', '方法', '新聞來源', '連結', '有撚用', '港台', '事件', '曱甴', '公眾', '利益', '凌駕', '私隱', '架咩', '快', '用', '健康碼', '才是', '根本', '唔使理', '照裝', '講明', '唔想裝', '既', '能裝', '死', '佬', '尋日', '去搵', '個隻', '狐狸精', '薛', '永恆', '食得屎', '唔用', '而係', '每行', '一步', '掃描', '二維碼', '一次', '掃描', '一樣', '有記錄', '唔會用', '中央', '共狗', '野', '唔會裝', '手動', '掃描', '唔記', '名', '無', '記錄', '鴨同', '無分別', '吊', '晒錢', '晒時間', '自欺欺人', '記錄', '晒', '既', '行蹤', '一開', '睇晒', '唔覺得', '好多人', '會用', '囉', '回', '鬼門關', '大開', '唔好', '將呢', '垃圾', '香港', '中共', '扯上', '關係', '影衰', '中共', '中共', '既', '防疫', '追蹤', '系統', '好過', '香港', '千倍', '唔駛', 

## ***3. Implement bigram and trigram to tokens***

In [10]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(result_fenci, min_count=10, threshold=10)  # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[result_fenci], min_count=10, threshold=10)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


def process_words(texts):
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]

    return texts


ngram_fenci = process_words(result_fenci)  # processed Text Data!

print(ngram_fenci[:10])

[['安心出行', '下周一', '出爐', '毋須', '記名', '及', '不', '使用', '記錄', '創新_及_科技_局局長', '薛_永恒', '宣布', '安心出行_應用程式', '下周一', '供_市民', '下載', '自行', '紀錄', '用家', '出行', '習慣', '方便', '日後', '追蹤', '減低', '社區_傳播', '風險', '用家', '進入', '參與_計劃', '的場', '安心出行', '下周一', '出爐', '毋須', '記名', '及', '不', '使用', '記錄', '希望', '市民', '齊心', '支持', '安心出行', '市民', '萬一出', '事', '也', '可讓', '政府部門', '追查', '源頭', '及', '相關', '地點', '這', '做法', '跟_內地', '也', '相同', '都是', '有效', '追查', '病者', '行蹤', '方法', '新聞來源_連結', '有撚用', '港台', '事件', '曱甴', '公眾_利益', '凌駕', '私隱', '架咩', '快', '用', '健康碼', '才是', '根本', '唔使理', '照裝', '講明', '唔想裝', '既', '能裝', '死', '佬', '尋日', '去搵', '個隻', '狐狸精', '薛', '永恆', '食得屎', '唔用', '而係', '每行', '一步', '掃描_二維碼', '一次', '掃描', '一樣', '有記錄', '唔會用', '中央', '共狗', '野', '唔會裝', '手動', '掃描', '唔記', '名', '無', '記錄', '鴨同', '無分別', '吊', '晒錢', '晒時間', '自欺欺人', '記錄', '晒', '既', '行蹤', '一開', '睇晒', '唔覺得', '好多人', '會用', '囉', '回', '鬼門關', '大開', '唔好', '將呢', '垃圾', '香港', '中共', '扯上', '關係', '影衰', '中共', '中共', '既', '防疫', '追蹤', '系統', '好過', '香港', '千倍', '唔駛', '審', '會話', '資料', '大陸', '了', '香港', '疫

In [11]:
df['tokenz'] = ngram_fenci
df

,ID,thread_id,thread_title,category,forum,stance,msg,reply_time,reply_date,day,content,tokenz
0,HD_29534436,29534436,安心出行app下周一出爐 毋須記名及不使用gps記錄,時事新聞,hkdiscuss,blue,創新及科技局局長薛永恒宣布，「安心出行」應用程式下周一供市民下載，自行紀錄用家出行習慣，方便...,2020-11-11 11:56,2020-11-11,1,安心出行app下周一出爐 毋須記名及不使用gps記錄創新及科技局局長薛永恒宣布 安心出行 ...,"[安心出行, 下周一, 出爐, 毋須, 記名, 及, 不, 使用, 記錄, 創新_及_科技_..."
1,HD_29534444,29534444,安心出行下周啟用助市民記錄行程場所爆疫會有通知,時事新聞,hkdiscuss,blue,"用唔用都可以照出入場所及乘坐交通工具，有咩人會用呀？,睇吓你去過邊度飲酒，邊度揼骨？,政府出...",2020-11-11 12:07,2020-11-11,1,安心出行下周啟用助市民記錄行程場所爆疫會有通知用唔用都可以照出入場所及乘坐交通工具 有咩人會...,"[安心出行, 下周, 啟用, 助, 市民, 記錄_行程, 場所, 爆疫, 會有, 通知, 用..."
2,HG_7327032,7327032,安心出行流動應用程式下周一起下載六千場地可使用,CA,hkgolden,yellow,"齊揮手.jpg ,十年前，唔會講｢出行｣又係支共文化入侵,想知個apps有乜用雖然都唔會do...",2020-11-11 12:29:14,2020-11-11,1,安心出行流動應用程式下周一起下載六千場地可使用齊揮手 jpg 十年前 唔會講 出行 又係支...,"[安心出行_流動_應用程式, 下周, 一起, 下載, 六千, 場地, 可使用, 齊, 揮手,..."
3,HG_7327159,7327159,港府下週一推安心出行app 稱屬自願參與用戶無須登記姓名,CA,hkgolden,yellow,"代言人,尖尖尖尖尖～～～～～尖尖尖～～～尖～～～,安心上路,如果doo一次有錢收一定大把人doo",2020-11-11 14:38:24,2020-11-11,1,港府下週一推安心出行app 稱屬自願參與用戶無須登記姓名代言人 尖尖尖尖尖 尖尖尖 ...,"[港府, 下週一, 推, 安心出行, 稱屬, 自願_參與, 用戶, 無須, 登記_姓名, 代..."
4,HD_29536981,29536981,政府推出安心出行流動應用程式,消閒娛樂,hkdiscuss,blue,政府今日（十一月十一日）公布推出感染風險通知「安心出行」流動應用程式（leavehomesa...,2020-11-12 02:13,2020-11-12,2,政府推出安心出行流動應用程式政府今日 十一月十一日 公布推出感染風險通知 安心出行 流動應用...,"[政府, 推出, 安心出行_流動_應用程式, 政府, 今日, 十一月, 十一日, 公布, 推..."
...,...,...,...,...,...,...,...,...,...,...,...,...
3689,BB_23578280,23578280,老人家安心出行手機,自由講場,babykingdom,blue,"老人家用唔慣智能電話， 好多時要撳好多次先撳到number, 撳好耐先打到電話用到發忟。 想...",2022-05-09 21:45:00,2022-05-09,545,老人家安心出行手機老人家用唔慣智能電話 好多時要撳好多次先撳到number 撳好耐先打到...,"[老人家, 安心出行, 手機, 老人家, 用, 唔慣, 智能_電話, 好多時, 撳, 多次,..."
3690,HG_7560717,7560717,而家係咪去酒店賓館爆房都要嘟安心打兩針,CA,hkgolden,yellow,"首先你要有仔, ,高登仔唔去爆房",2022-05-09 12:26:34,2022-05-09,545,而家係咪去酒店賓館爆房都要嘟安心打兩針首先你要有仔 高登仔唔去爆房,"[係咪, 酒店賓館, 爆房, 都要_嘟, 安心, 打_兩針, 首先, 有仔, 高登仔, 唔去..."
3691,BB_23578547,23578547,如果今日有更新個安心出行,自由講場,babykingdom,blue,"你哋盡快check下你哋個針卡仲喺唔喺度 ， 因為今日好多人都冇咗個針卡要重新入過。,還在無...",2022-05-10 21:16:00,2022-05-10,546,如果今日有更新個安心出行你哋盡快check下你哋個針卡仲喺唔喺度 因為今日好多人都冇咗個...,"[今日, 更新, 安心出行, 盡快, 下, 針卡, 仲, 唔喺度, 今日, 好多人, 冇咗,..."
3692,BB_23578570,23578570,緊急 安心出行更新,自由講場,babykingdom,blue,andro機，3.2.5更新版本 會無咗針咭紀錄 裝咗醫健通嘅可以滙入 無裝嘅最好影低針紙個...,2022-05-10 23:04:00,2022-05-10,546,緊急 安心出行更新 andro機 更新版本 會無咗針咭紀錄 裝咗醫健通嘅可以滙入 無裝嘅...,"[緊急, 安心出行, 更新, 機, 更新_版本, 無咗, 針咭, 紀錄, 裝咗, 醫健, 通..."


## ***4. Removal of unnecessary symbols***

In [12]:
def sent_to_words_space(sentences):
    for sent in sentences:
        sent = re.sub(r'[^\w\s]', '', sent)  # remove punctuations
        sent = sent.replace('nan', '')
        sent = sent.replace('_', '')
        yield (sent)


df['tokenz'] = df['tokenz'].astype(str)

msg_space = df.tokenz.values.tolist()
msg_space_words = list(sent_to_words_space(msg_space))

print(msg_space_words[:1])

df['tokenz'] = msg_space_words
df

['安心出行 下周一 出爐 毋須 記名 及 不 使用 記錄 創新及科技局局長 薛永恒 宣布 安心出行應用程式 下周一 供市民 下載 自行 紀錄 用家 出行 習慣 方便 日後 追蹤 減低 社區傳播 風險 用家 進入 參與計劃 的場 安心出行 下周一 出爐 毋須 記名 及 不 使用 記錄 希望 市民 齊心 支持 安心出行 市民 萬一出 事 也 可讓 政府部門 追查 源頭 及 相關 地點 這 做法 跟內地 也 相同 都是 有效 追查 病者 行蹤 方法 新聞來源連結 有撚用 港台 事件 曱甴 公眾利益 凌駕 私隱 架咩 快 用 健康碼 才是 根本 唔使理 照裝 講明 唔想裝 既 能裝 死 佬 尋日 去搵 個隻 狐狸精 薛 永恆 食得屎 唔用 而係 每行 一步 掃描二維碼 一次 掃描 一樣 有記錄 唔會用 中央 共狗 野 唔會裝 手動 掃描 唔記 名 無 記錄 鴨同 無分別 吊 晒錢 晒時間 自欺欺人 記錄 晒 既 行蹤 一開 睇晒 唔覺得 好多人 會用 囉 回 鬼門關 大開 唔好 將呢 垃圾 香港 中共 扯上 關係 影衰 中共 中共 既 防疫 追蹤 系統 好過 香港 千倍 唔駛 審 會話 資料 大陸 了 香港 疫情 冇得斷 尾 好大 原因 是 黃暴 政治 乃 共 好丫 起碼 中 病毒 乃西 慘啦 弗系 抗疫 仲要 自帶 狗糧 用完即棄 冇得斷 武漢 放毒 你明唔明 健康碼 加強 效用 就要 加入 安心出行 呢種 功能 武漢 放毒 肺炎 正仆街 講乜 唔會改 變 中國 抗疫成功 老美 日日 創新高 呢個 事實 閣下 只有 技倆 恕 失陪 對 化膿 功 膠登 腦殘 果類 廢 已 免疫 武漢肺炎 全世界人 討厭 中國 全世界 唔like 中國 唯獨 只有 港澳區 市民 永遠 不行 水 糧 電 日用品 要靠 內地 同胞 生產 送來 唔好 再講 比錢 去買 我們 香港人 什麼 資格 偏低 或 討厭 國家 自知 量力 唔好 再裝 大香港 人講 又講 咁冇腦 嘥鳩氣 自願性 手動 掃描 當事人 選擇性 掃 行蹤記錄 和 有心 隱瞞 口述 什麼 分別 政府 高官 始終 改 唔到 除褲放屁 思維 唔知 搵 許志安黃心穎 做 宣傳 安心出行 相信 唔包 曱甴 仆街 美國 肺炎 害 全球 大陸 飲 紐西蘭 荷蘭 奶粉 外國 送來 讓 大 唔好 再講 比錢 去買 中國人 有咩資格 唔飲 奶 思源 

,ID,thread_id,thread_title,category,forum,stance,msg,reply_time,reply_date,day,content,tokenz
0,HD_29534436,29534436,安心出行app下周一出爐 毋須記名及不使用gps記錄,時事新聞,hkdiscuss,blue,創新及科技局局長薛永恒宣布，「安心出行」應用程式下周一供市民下載，自行紀錄用家出行習慣，方便...,2020-11-11 11:56,2020-11-11,1,安心出行app下周一出爐 毋須記名及不使用gps記錄創新及科技局局長薛永恒宣布 安心出行 ...,安心出行 下周一 出爐 毋須 記名 及 不 使用 記錄 創新及科技局局長 薛永恒 宣布 安心...
1,HD_29534444,29534444,安心出行下周啟用助市民記錄行程場所爆疫會有通知,時事新聞,hkdiscuss,blue,"用唔用都可以照出入場所及乘坐交通工具，有咩人會用呀？,睇吓你去過邊度飲酒，邊度揼骨？,政府出...",2020-11-11 12:07,2020-11-11,1,安心出行下周啟用助市民記錄行程場所爆疫會有通知用唔用都可以照出入場所及乘坐交通工具 有咩人會...,安心出行 下周 啟用 助 市民 記錄行程 場所 爆疫 會有 通知 用唔用 照 出入 場所 及...
2,HG_7327032,7327032,安心出行流動應用程式下周一起下載六千場地可使用,CA,hkgolden,yellow,"齊揮手.jpg ,十年前，唔會講｢出行｣又係支共文化入侵,想知個apps有乜用雖然都唔會do...",2020-11-11 12:29:14,2020-11-11,1,安心出行流動應用程式下周一起下載六千場地可使用齊揮手 jpg 十年前 唔會講 出行 又係支...,安心出行流動應用程式 下周 一起 下載 六千 場地 可使用 齊 揮手 十年前 出行 又係 支...
3,HG_7327159,7327159,港府下週一推安心出行app 稱屬自願參與用戶無須登記姓名,CA,hkgolden,yellow,"代言人,尖尖尖尖尖～～～～～尖尖尖～～～尖～～～,安心上路,如果doo一次有錢收一定大把人doo",2020-11-11 14:38:24,2020-11-11,1,港府下週一推安心出行app 稱屬自願參與用戶無須登記姓名代言人 尖尖尖尖尖 尖尖尖 ...,港府 下週一 推 安心出行 稱屬 自願參與 用戶 無須 登記姓名 代言人 尖尖 尖尖 尖 尖...
4,HD_29536981,29536981,政府推出安心出行流動應用程式,消閒娛樂,hkdiscuss,blue,政府今日（十一月十一日）公布推出感染風險通知「安心出行」流動應用程式（leavehomesa...,2020-11-12 02:13,2020-11-12,2,政府推出安心出行流動應用程式政府今日 十一月十一日 公布推出感染風險通知 安心出行 流動應用...,政府 推出 安心出行流動應用程式 政府 今日 十一月 十一日 公布 推出 感染風險通知 安心...
...,...,...,...,...,...,...,...,...,...,...,...,...
3689,BB_23578280,23578280,老人家安心出行手機,自由講場,babykingdom,blue,"老人家用唔慣智能電話， 好多時要撳好多次先撳到number, 撳好耐先打到電話用到發忟。 想...",2022-05-09 21:45:00,2022-05-09,545,老人家安心出行手機老人家用唔慣智能電話 好多時要撳好多次先撳到number 撳好耐先打到...,老人家 安心出行 手機 老人家 用 唔慣 智能電話 好多時 撳 多次 先 撳 撳 好耐 先 ...
3690,HG_7560717,7560717,而家係咪去酒店賓館爆房都要嘟安心打兩針,CA,hkgolden,yellow,"首先你要有仔, ,高登仔唔去爆房",2022-05-09 12:26:34,2022-05-09,545,而家係咪去酒店賓館爆房都要嘟安心打兩針首先你要有仔 高登仔唔去爆房,係咪 酒店賓館 爆房 都要嘟 安心 打兩針 首先 有仔 高登仔 唔去 爆房
3691,BB_23578547,23578547,如果今日有更新個安心出行,自由講場,babykingdom,blue,"你哋盡快check下你哋個針卡仲喺唔喺度 ， 因為今日好多人都冇咗個針卡要重新入過。,還在無...",2022-05-10 21:16:00,2022-05-10,546,如果今日有更新個安心出行你哋盡快check下你哋個針卡仲喺唔喺度 因為今日好多人都冇咗個...,今日 更新 安心出行 盡快 下 針卡 仲 唔喺度 今日 好多人 冇咗 針卡要 重新 入過 還...
3692,BB_23578570,23578570,緊急 安心出行更新,自由講場,babykingdom,blue,andro機，3.2.5更新版本 會無咗針咭紀錄 裝咗醫健通嘅可以滙入 無裝嘅最好影低針紙個...,2022-05-10 23:04:00,2022-05-10,546,緊急 安心出行更新 andro機 更新版本 會無咗針咭紀錄 裝咗醫健通嘅可以滙入 無裝嘅...,緊急 安心出行 更新 機 更新版本 無咗 針咭 紀錄 裝咗 醫健 通 滙 入 無 裝 最好 ...


## ***5. Word count and stop word filtering for tokenz column***

In [13]:
df['tokenz'] = df['tokenz'].astype(str)
data = df['tokenz'].values.tolist()
print("Total Number of Documents:", len(data))

Total Number of Documents: 3694


In [14]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = gensim.utils.simple_preprocess(str(sent), min_len=2, deacc=True)
        yield sent

data_words = list(sent_to_words(data))
print(data_words[:5])

[['安心出行', '下周一', '出爐', '毋須', '記名', '使用', '記錄', '創新及科技局局長', '薛永恒', '宣布', '安心出行應用程式', '下周一', '供市民', '下載', '自行', '紀錄', '用家', '出行', '習慣', '方便', '日後', '追蹤', '減低', '社區傳播', '風險', '用家', '進入', '參與計劃', '的場', '安心出行', '下周一', '出爐', '毋須', '記名', '使用', '記錄', '希望', '市民', '齊心', '支持', '安心出行', '市民', '萬一出', '可讓', '政府部門', '追查', '源頭', '相關', '地點', '做法', '跟內地', '相同', '都是', '有效', '追查', '病者', '行蹤', '方法', '新聞來源連結', '有撚用', '港台', '事件', '曱甴', '公眾利益', '凌駕', '私隱', '架咩', '健康碼', '才是', '根本', '唔使理', '照裝', '講明', '唔想裝', '能裝', '尋日', '去搵', '個隻', '狐狸精', '永恆', '食得屎', '唔用', '而係', '每行', '一步', '掃描二維碼', '一次', '掃描', '一樣', '有記錄', '唔會用', '中央', '共狗', '唔會裝', '手動', '掃描', '唔記', '記錄', '鴨同', '無分別', '晒錢', '晒時間', '自欺欺人', '記錄', '行蹤', '一開', '睇晒', '唔覺得', '好多人', '會用', '鬼門關', '大開', '唔好', '將呢', '垃圾', '香港', '中共', '扯上', '關係', '影衰', '中共', '中共', '防疫', '追蹤', '系統', '好過', '香港', '千倍', '唔駛', '會話', '資料', '大陸', '香港', '疫情', '冇得斷', '好大', '原因', '黃暴', '政治', '好丫', '起碼', '病毒', '乃西', '慘啦', '弗系', '抗疫', '仲要', '自帶', '狗糧', '用完即棄', '冇得斷', '武漢', '放毒', '你明唔明', '健康碼', '加強',

In [15]:
stop_words = [line.strip() for line in open('dict/stopwords.txt', 'r', encoding='UTF-8').readlines()]
POSdict = [line.strip() for line in open('dict/POSdict.txt', 'r', encoding='UTF-8').readlines()]
canton = [line.strip() for line in open('dict/canton.txt', 'r', encoding='UTF-8').readlines()]

def process_words(texts, stop_words=stop_words):
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [[word for word in simple_preprocess(str(doc)) if word not in POSdict] for doc in texts]
    texts = [[word for word in simple_preprocess(str(doc)) if word not in canton] for doc in texts]
    return texts

data_ready = process_words(data_words)
print(data_ready[:5])

df['tokenz'] = data_ready

[['安心出行', '下周一', '出爐', '記名', '記錄', '創新及科技局局長', '薛永恒', '宣布', '安心出行應用程式', '下周一', '供市民', '紀錄', '用家', '出行', '習慣', '追蹤', '減低', '社區傳播', '風險', '用家', '參與計劃', '的場', '安心出行', '下周一', '出爐', '記名', '記錄', '市民', '齊心', '支持', '安心出行', '市民', '可讓', '政府部門', '追查', '源頭', '地點', '做法', '跟內地', '追查', '病者', '行蹤', '方法', '有撚用', '曱甴', '公眾利益', '凌駕', '私隱', '架咩', '健康碼', '唔使理', '照裝', '講明', '唔想裝', '能裝', '去搵', '個隻', '狐狸精', '永恆', '食得屎', '掃描二維碼', '掃描', '有記錄', '唔會用', '中央', '共狗', '唔會裝', '手動', '掃描', '唔記', '記錄', '鴨同', '無分別', '晒錢', '晒時間', '自欺欺人', '記錄', '行蹤', '睇晒', '唔覺得', '好多人', '鬼門關', '大開', '將呢', '垃圾', '中共', '扯上', '關係', '影衰', '中共', '中共', '防疫', '追蹤', '系統', '唔駛', '會話', '資料', '大陸', '疫情', '冇得斷', '原因', '黃暴', '政治', '好丫', '病毒', '乃西', '慘啦', '弗系', '抗疫', '自帶', '狗糧', '用完即棄', '冇得斷', '武漢', '放毒', '你明唔明', '健康碼', '效用', '安心出行', '功能', '武漢', '放毒', '肺炎', '正仆街', '講乜', '唔會改', '中國', '抗疫成功', '老美', '創新高', '事實', '閣下', '技倆', '失陪', '化膿', '膠登', '腦殘', '果類', '免疫', '武漢肺炎', '全世界人', '討厭', '中國', '全世界', '唔like', '中國', '港澳區', '市民', '不行', '日用品', '要靠', '內地', '同胞', '生產',

In [16]:
def sent_to_words_space(sentences):
    for sent in sentences:
        sent = re.sub(r'[^\w\s]', '', sent)  # remove punctuations
        yield sent

df['tokenz'] = df['tokenz'].astype(str)

msg_space = df.tokenz.values.tolist()
msg_space_words = list(sent_to_words_space(msg_space))

print(msg_space_words[:1])

df['tokenz'] = msg_space_words
df

['安心出行 下周一 出爐 記名 記錄 創新及科技局局長 薛永恒 宣布 安心出行應用程式 下周一 供市民 紀錄 用家 出行 習慣 追蹤 減低 社區傳播 風險 用家 參與計劃 的場 安心出行 下周一 出爐 記名 記錄 市民 齊心 支持 安心出行 市民 可讓 政府部門 追查 源頭 地點 做法 跟內地 追查 病者 行蹤 方法 有撚用 曱甴 公眾利益 凌駕 私隱 架咩 健康碼 唔使理 照裝 講明 唔想裝 能裝 去搵 個隻 狐狸精 永恆 食得屎 掃描二維碼 掃描 有記錄 唔會用 中央 共狗 唔會裝 手動 掃描 唔記 記錄 鴨同 無分別 晒錢 晒時間 自欺欺人 記錄 行蹤 睇晒 唔覺得 好多人 鬼門關 大開 將呢 垃圾 中共 扯上 關係 影衰 中共 中共 防疫 追蹤 系統 唔駛 會話 資料 大陸 疫情 冇得斷 原因 黃暴 政治 好丫 病毒 乃西 慘啦 弗系 抗疫 自帶 狗糧 用完即棄 冇得斷 武漢 放毒 你明唔明 健康碼 效用 安心出行 功能 武漢 放毒 肺炎 正仆街 講乜 唔會改 中國 抗疫成功 老美 創新高 事實 閣下 技倆 失陪 化膿 膠登 腦殘 果類 免疫 武漢肺炎 全世界人 討厭 中國 全世界 唔like 中國 港澳區 市民 不行 日用品 要靠 內地 同胞 生產 送來 比錢 去買 香港人 資格 偏低 討厭 國家 自知 量力 再裝 大香港 人講 又講 咁冇腦 嘥鳩氣 自願性 手動 掃描 當事人 選擇性 行蹤記錄 有心 隱瞞 口述 政府 高官 除褲放屁 思維 許志安黃心穎 宣傳 安心出行 唔包 曱甴 仆街 美國 肺炎 全球 大陸 紐西蘭 荷蘭 奶粉 外國 送來 比錢 去買 中國人 有咩資格 唔飲 思源 個啲 養大 大陸人 電腦 軟件 美國人 研發 送來 要俾錢 去買 中國人 資格 偏低 討厭 美國']


,ID,thread_id,thread_title,category,forum,stance,msg,reply_time,reply_date,day,content,tokenz
0,HD_29534436,29534436,安心出行app下周一出爐 毋須記名及不使用gps記錄,時事新聞,hkdiscuss,blue,創新及科技局局長薛永恒宣布，「安心出行」應用程式下周一供市民下載，自行紀錄用家出行習慣，方便...,2020-11-11 11:56,2020-11-11,1,安心出行app下周一出爐 毋須記名及不使用gps記錄創新及科技局局長薛永恒宣布 安心出行 ...,安心出行 下周一 出爐 記名 記錄 創新及科技局局長 薛永恒 宣布 安心出行應用程式 下周一...
1,HD_29534444,29534444,安心出行下周啟用助市民記錄行程場所爆疫會有通知,時事新聞,hkdiscuss,blue,"用唔用都可以照出入場所及乘坐交通工具，有咩人會用呀？,睇吓你去過邊度飲酒，邊度揼骨？,政府出...",2020-11-11 12:07,2020-11-11,1,安心出行下周啟用助市民記錄行程場所爆疫會有通知用唔用都可以照出入場所及乘坐交通工具 有咩人會...,安心出行 啟用 市民 記錄行程 場所 爆疫 通知 用唔用 出入 場所 乘坐 交通工具 咩人 ...
2,HG_7327032,7327032,安心出行流動應用程式下周一起下載六千場地可使用,CA,hkgolden,yellow,"齊揮手.jpg ,十年前，唔會講｢出行｣又係支共文化入侵,想知個apps有乜用雖然都唔會do...",2020-11-11 12:29:14,2020-11-11,1,安心出行流動應用程式下周一起下載六千場地可使用齊揮手 jpg 十年前 唔會講 出行 又係支...,安心出行流動應用程式 場地 可使用 揮手 十年前 出行 支共 文化 入侵 有乜用 有乜用 啟...
3,HG_7327159,7327159,港府下週一推安心出行app 稱屬自願參與用戶無須登記姓名,CA,hkgolden,yellow,"代言人,尖尖尖尖尖～～～～～尖尖尖～～～尖～～～,安心上路,如果doo一次有錢收一定大把人doo",2020-11-11 14:38:24,2020-11-11,1,港府下週一推安心出行app 稱屬自願參與用戶無須登記姓名代言人 尖尖尖尖尖 尖尖尖 ...,港府 下週一 安心出行 稱屬 自願參與 用戶 登記姓名 代言人 尖尖 尖尖 尖尖 安心上路 ...
4,HD_29536981,29536981,政府推出安心出行流動應用程式,消閒娛樂,hkdiscuss,blue,政府今日（十一月十一日）公布推出感染風險通知「安心出行」流動應用程式（leavehomesa...,2020-11-12 02:13,2020-11-12,2,政府推出安心出行流動應用程式政府今日 十一月十一日 公布推出感染風險通知 安心出行 流動應用...,政府 推出 安心出行流動應用程式 政府 公布 推出 感染風險通知 安心出行流動應用程式 旨在...
...,...,...,...,...,...,...,...,...,...,...,...,...
3689,BB_23578280,23578280,老人家安心出行手機,自由講場,babykingdom,blue,"老人家用唔慣智能電話， 好多時要撳好多次先撳到number, 撳好耐先打到電話用到發忟。 想...",2022-05-09 21:45:00,2022-05-09,545,老人家安心出行手機老人家用唔慣智能電話 好多時要撳好多次先撳到number 撳好耐先打到...,老人家 安心出行 手機 老人家 唔慣 智能電話 好多時 電話 買個 部新 舊式 號碼 撳掣 ...
3690,HG_7560717,7560717,而家係咪去酒店賓館爆房都要嘟安心打兩針,CA,hkgolden,yellow,"首先你要有仔, ,高登仔唔去爆房",2022-05-09 12:26:34,2022-05-09,545,而家係咪去酒店賓館爆房都要嘟安心打兩針首先你要有仔 高登仔唔去爆房,酒店賓館 爆房 都要嘟 打兩針 有仔 唔去 爆房
3691,BB_23578547,23578547,如果今日有更新個安心出行,自由講場,babykingdom,blue,"你哋盡快check下你哋個針卡仲喺唔喺度 ， 因為今日好多人都冇咗個針卡要重新入過。,還在無...",2022-05-10 21:16:00,2022-05-10,546,如果今日有更新個安心出行你哋盡快check下你哋個針卡仲喺唔喺度 因為今日好多人都冇咗個...,安心出行 針卡 唔喺度 好多人 冇咗 針卡要 入過 還在 無問題 仔入 餐廳 無左 針卡 出...
3692,BB_23578570,23578570,緊急 安心出行更新,自由講場,babykingdom,blue,andro機，3.2.5更新版本 會無咗針咭紀錄 裝咗醫健通嘅可以滙入 無裝嘅最好影低針紙個...,2022-05-10 23:04:00,2022-05-10,546,緊急 安心出行更新 andro機 更新版本 會無咗針咭紀錄 裝咗醫健通嘅可以滙入 無裝嘅...,安心出行 更新版本 無咗 針咭 紀錄 裝咗 醫健 影低 針紙 以備不時 中學生 放學 要堂食...


In [17]:
df.to_csv("results/PP_leavehomesafe.csv", encoding='utf-8', index=False)